In [1]:
# AWS Glue Catalog and Athena Tables Creation
# Este script crea las definiciones de tablas en AWS Glue para los datos económicos
# subidos al bucket S3, permitiendo consultas con Amazon Athena

import boto3
import time
from botocore.exceptions import ClientError

In [2]:
# Definimos las constantes
BUCKET_NAME = "itam-analytics-ragp"
S3_RAW_DIRECTORY = "raw/"
PROFILE_NAME = "datascientist"
DATABASE_NAME = "econ"


In [3]:
def conectar_glue():
    """
    Establece conexión con AWS Glue y Athena usando el perfil especificado.
    """
    try:
        # Creamos una sesión usando el perfil
        session = boto3.Session(profile_name=PROFILE_NAME)
        
        # Creamos el cliente Glue
        glue_client = session.client('glue')
        
        # Creamos el cliente Athena
        athena_client = session.client('athena')
        
        print(f"✅ Conexión establecida con AWS Glue y Athena.")
        return glue_client, athena_client
    
    except Exception as e:
        print(f"❌ Error al conectar con AWS: {str(e)}")
        return None, None


In [4]:
def crear_base_datos(glue_client):
    """
    Crea la base de datos en AWS Glue Catalog si no existe.
    """
    try:
        # Verificamos si la base de datos ya existe
        try:
            glue_client.get_database(Name=DATABASE_NAME)
            print(f"ℹ️ Base de datos '{DATABASE_NAME}' ya existe.")
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'EntityNotFoundException':
                # Creamos la base de datos
                glue_client.create_database(
                    DatabaseInput={
                        'Name': DATABASE_NAME,
                        'Description': 'Indicadores económicos de México: tipo de cambio, inflación y tasas de interés',
                    }
                )
                print(f"✅ Base de datos '{DATABASE_NAME}' creada exitosamente.")
                return True
            else:
                raise
    
    except Exception as e:
        print(f"❌ Error al crear la base de datos: {str(e)}")
        return False


In [5]:
def crear_tabla_tipo_cambio(glue_client):
    """
    Crea la tabla para los datos de tipo de cambio.
    """
    tabla_nombre = "tipo_cambio"
    s3_ruta = f"s3://{BUCKET_NAME}/{S3_RAW_DIRECTORY}{tabla_nombre}"
    
    try:
        # Verificamos si la tabla ya existe
        try:
            glue_client.get_table(DatabaseName=DATABASE_NAME, Name=tabla_nombre)
            print(f"ℹ️ Tabla '{tabla_nombre}' ya existe.")
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'EntityNotFoundException':
                # Configuración de la tabla
                tabla_input = {
                    'Name': tabla_nombre,
                    'Description': 'Tipo de cambio FIX (peso/dólar) de Banxico',
                    'StorageDescriptor': {
                        'Columns': [
                            {'Name': 'tipo_cambio', 'Type': 'double'},
                            {'Name': 'fecha', 'Type': 'date'}
                        ],
                        'Location': s3_ruta,
                        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
                        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
                        'SerdeInfo': {
                            'SerializationLibrary': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe',
                            'Parameters': {
                                'field.delim': ','
                            }
                        }
                    },
                    'TableType': 'EXTERNAL',
                    'Parameters': {
                        'classification': 'csv',
                        'skip.header.line.count': '1'
                    }
                }
                
                # Creamos la tabla
                glue_client.create_table(
                    DatabaseName=DATABASE_NAME,
                    TableInput=tabla_input
                )
                
                print(f"✅ Tabla '{tabla_nombre}' creada exitosamente.")
                return True
            else:
                raise
    
    except Exception as e:
        print(f"❌ Error al crear la tabla '{tabla_nombre}': {str(e)}")
        return False

In [6]:
def crear_tabla_inflacion(glue_client):
    """
    Crea la tabla para los datos de inflación.
    """
    tabla_nombre = "inflacion"
    s3_ruta = f"s3://{BUCKET_NAME}/{S3_RAW_DIRECTORY}{tabla_nombre}"
    
    try:
        # Verificamos si la tabla ya existe
        try:
            glue_client.get_table(DatabaseName=DATABASE_NAME, Name=tabla_nombre)
            print(f"ℹ️ Tabla '{tabla_nombre}' ya existe.")
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'EntityNotFoundException':
                # Configuración de la tabla
                tabla_input = {
                    'Name': tabla_nombre,
                    'Description': 'Datos de inflación mensual (INPC) del INEGI',
                    'StorageDescriptor': {
                        'Columns': [
                            {'Name': 'fecha', 'Type': 'date'},
                            {'Name': 'inflacion_mensual', 'Type': 'double'}
                        ],
                        'Location': s3_ruta,
                        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
                        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
                        'SerdeInfo': {
                            'SerializationLibrary': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe',
                            'Parameters': {
                                'field.delim': ','
                            }
                        }
                    },
                    'TableType': 'EXTERNAL',
                    'Parameters': {
                        'classification': 'csv',
                        'skip.header.line.count': '1'
                    }
                }
                
                # Creamos la tabla
                glue_client.create_table(
                    DatabaseName=DATABASE_NAME,
                    TableInput=tabla_input
                )
                
                print(f"✅ Tabla '{tabla_nombre}' creada exitosamente.")
                return True
            else:
                raise
    
    except Exception as e:
        print(f"❌ Error al crear la tabla '{tabla_nombre}': {str(e)}")
        return False


In [7]:

def crear_tabla_tasas_interes(glue_client):
    """
    Crea la tabla para los datos de tasas de interés.
    """
    tabla_nombre = "tasas_interes"
    s3_ruta = f"s3://{BUCKET_NAME}/{S3_RAW_DIRECTORY}{tabla_nombre}"
    
    try:
        # Verificamos si la tabla ya existe
        try:
            glue_client.get_table(DatabaseName=DATABASE_NAME, Name=tabla_nombre)
            print(f"ℹ️ Tabla '{tabla_nombre}' ya existe.")
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'EntityNotFoundException':
                # Configuración de la tabla
                tabla_input = {
                    'Name': tabla_nombre,
                    'Description': 'Tasas de interés CETES 28 días de Banxico',
                    'StorageDescriptor': {
                        'Columns': [
                            {'Name': 'tasa_cetes28', 'Type': 'double'},
                            {'Name': 'fecha', 'Type': 'date'}
                        ],
                        'Location': s3_ruta,
                        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
                        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
                        'SerdeInfo': {
                            'SerializationLibrary': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe',
                            'Parameters': {
                                'field.delim': ','
                            }
                        }
                    },
                    'TableType': 'EXTERNAL',
                    'Parameters': {
                        'classification': 'csv',
                        'skip.header.line.count': '1'
                    }
                }
                
                # Creamos la tabla
                glue_client.create_table(
                    DatabaseName=DATABASE_NAME,
                    TableInput=tabla_input
                )
                
                print(f"✅ Tabla '{tabla_nombre}' creada exitosamente.")
                return True
            else:
                raise
    
    except Exception as e:
        print(f"❌ Error al crear la tabla '{tabla_nombre}': {str(e)}")
        return False


In [10]:
"""
Función principal que orquesta la creación de bases de datos y tablas.
"""
print("\n🔍 Iniciando creación de catálogo y tablas en AWS Glue/Athena")

# Conectamos con Glue y Athena
glue_client, athena_client = conectar_glue()
if not glue_client or not athena_client:
    print("❌ No se pudo establecer conexión con AWS. Abortando proceso.")



🔍 Iniciando creación de catálogo y tablas en AWS Glue/Athena
✅ Conexión establecida con AWS Glue y Athena.


In [11]:

# Creamos la base de datos
if not crear_base_datos(glue_client):
    print("❌ No se pudo crear la base de datos. Abortando proceso.")

    

✅ Base de datos 'econ' creada exitosamente.


In [12]:
# Creamos las tablas
exito_tipo_cambio = crear_tabla_tipo_cambio(glue_client)
exito_inflacion = crear_tabla_inflacion(glue_client)
exito_tasas = crear_tabla_tasas_interes(glue_client)

if exito_tipo_cambio and exito_inflacion and exito_tasas:
    print("\n✅ Todas las tablas han sido creadas exitosamente.")

✅ Tabla 'tipo_cambio' creada exitosamente.
✅ Tabla 'inflacion' creada exitosamente.
✅ Tabla 'tasas_interes' creada exitosamente.

✅ Todas las tablas han sido creadas exitosamente.
